# Práctica 2: Patrones secuenciales

Grupo 8: Marina Garrido y María Montero

19/03/2024

El conjunto de datos “Online Retail.xlsx” contiene productos (definidos por la columna StockCode y Description) comprados por diferentes clientes (definidos por la columna CustomerID) en diferentes momentos (definidos por el campo InvoiceDate).

## Preprocesamiento de los datos

Se importan las librerías necesarias para la realización de la práctica.

In [ ]:
pip install gsppy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gsppy.gsp import GSP

Se importa el archivo de datos.

In [ ]:
data = pd.read_excel('Online Retail.xlsx')

print(data.shape)
data.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Se eliminan elementos repetidos y se estudia la presencia de elementos vacíos en el conjunto.

In [ ]:
data = data.drop_duplicates()

data.isna().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

Como no hay elementos vacíos en las columnas principales se puede continuar con el estudio de los posibles outliers que presente el conjunto de datos.

Los outliers son aquellos valores menores o iguales a 0, ya que no aportan información real al análisis, y aquellos valores por encima del valor máximo calculado.

In [ ]:
# Outliers por debajo del 0
data = data[data['Quantity'] > 0]

# Outliers por encima del
q1 = data['Quantity'].describe()[4]
q3 = data['Quantity'].describe()[6]

max_out = data.loc[data['Quantity'] > q3 + 1.5 * (q3 - q1), 'Quantity']

data = data[data['Quantity'] <= min(max_out)]

Se seleccionan las columnas principales para el análisis que se va a llevar a cabo.

In [ ]:
data = data[['CustomerID', 'InvoiceDate', 'StockCode']]

Eliminados estos datos, los registros de las compras quedarían de la siguiente manera.

In [ ]:
print(data.shape)
data.head()

(498811, 3)


,CustomerID,InvoiceDate,StockCode
0,17850,2010-12-01 08:26:00,85123A
1,17850,2010-12-01 08:26:00,71053
2,17850,2010-12-01 08:26:00,84406B
3,17850,2010-12-01 08:26:00,84029G
4,17850,2010-12-01 08:26:00,84029E


## 1.- Crear lista de secuencias

Se definen las transacciones separadas de cada cliente.

In [ ]:
transacciones = data.groupby(['CustomerID', 'InvoiceDate'])['StockCode'].apply(list)
transacciones = transacciones.reset_index()

In [ ]:
print(transacciones.shape)
transacciones.head(3)

(18998, 3)


,CustomerID,InvoiceDate,StockCode
0,12347,2010-12-07 14:57:00,"[85116, 22375, 71477, 22771, 22772, 22773, 227..."
1,12347,2011-01-26 14:30:00,"[84625A, 84625C, 85116, 20719, 22375, 22376, 2..."
2,12347,2011-04-07 10:43:00,"[22376, 22374, 22371, 22375, 20665, 21791, 225..."


Se agrupan estas transacciones por clientes para obtener las secuencias de compra de cada cliente.

In [ ]:
secuencias = transacciones.groupby('CustomerID')['StockCode'].apply(list)
secuencias = secuencias.reset_index()

In [ ]:
print(secuencias.shape)
secuencias.head(3)

(4238, 2)


,CustomerID,StockCode
0,12347,"[[85116, 22375, 71477, 22771, 22772, 22773, 22..."
1,12348,"[[84991, 21213, 22952, 21977, POST], [21726, P..."
2,12349,"[[23112, 23460, 21564, 21411, 21563, 22131, 22..."


## 2.- Estudio de los patrones de compra

Se aplica el algoritmo GSP con valores de soporte 0.5 y 0.1 para estudiar las diferencias en los patrones generados.

In [ ]:
gsp = GSP(secuencias[['StockCode']])

In [ ]:
soporte05 = gsp.search(0.5)
soporte05

In [ ]:
soporte08 = gsp.search(0.1)
soporte08

No se ha podido realizar esta última parte porque el tiempo de espera superaba la hora.